<a href="https://colab.research.google.com/github/aadittambe/actions-pipeline/blob/main/usgs_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import pandas as pd # import pandas library for data manipulation and analysis

# Import and clean data from GitHub
This code chunk imports the usgs_main.csv from the repo. It then cleans the data by taking the date column and separating it into the following columns: 

  - date: year-month-day format
  - time: the time of the earthquake in 12 hour format 
  - military_time: the time of the earthquake in 24 hour format



In [1]:


# Read in data
df_main = pd.read_csv('https://raw.githubusercontent.com/aadittambe/actions-pipeline/main/usgs_main.csv', index_col=None) # Enter the raw url from your repository

# Clean data
df_main["date_time"] = pd.to_datetime(df_main["time"]) # Convert time to a column called date_time
df_main.drop("time", axis = 1) # Drop the old time column

df_main = df_main.assign(   
    date = df_main["date_time"].dt.date, # Make new column with date in the format year-month-day
    time = df_main["date_time"].dt.strftime('%I:%M %p'), # Make new column with 12 hour format
    military_time = df_main["date_time"].dt.time # Make new colum with 24 hour format
    )

# Create variables to reference for the analysis sentence
This code chunk filters the usg_main.csv csv to calculate:
- the number of earthquakes in the dataframe
- the earliest earthquake that occured in the dataframe
- the latest earthquake that occured in the dataframe 
- the strongest earthquake that occured in the dataframe  

In [2]:
# Query the dataframe to isolate earthquakes to write about in a sentence
number_earthquakes = df_main.shape[0] # Return number of rows of dataframe
earliest = df_main[df_main["date_time"] == df_main["date_time"].min()]
latest = df_main[df_main["date_time"] == df_main["date_time"].max()]  # Return the row with the earliest earthquake since you started recording
strongest = df_main[df_main["mag"] == df_main["mag"].max()] # Return the row with the strongest earthquakes since you started recording

# Write a dynamically updating analysis sentence

This code chunk injects the variables constructed above into an string that updates with the latest variables

In [3]:
# Paste the values into a sentence. If there are earthquakes that happened at the same earliest time or had the same magnitude, we are taking the first row
print(f'Since {earliest.iloc[0]["time"]} on {earliest.iloc[0]["date"].strftime("%m/%d/%Y")} there have been {number_earthquakes} recorded earthquakes. The most recent earthquake was {latest.iloc[0]["mag"]} in magnitude and occured in/near {latest.iloc[0]["place"]}. The strongest earthquakes since the start of this webscraper was {strongest.iloc[0]["mag"]} magnitude and located in/near {strongest.iloc[0]["place"]}.')

Since 03:31 PM on 03/01/2022 there have been 458 recorded earthquakes. The most recent earthquake was 0.8 in magnitude and occured in/near Southern Alaska. The strongest earthquakes since the start of this webscraper was 6.6 magnitude and located in/near Kermadec Islands, New Zealand.
